In [ ]:
# Librerias

# Manipulacion 
import pandas as pd 
import geopandas as gpd
import re
from datetime import datetime

# Earth engine (descarga de imagenes)
import ee 

# Mapas interactivos
import folium

# Plot
from IPython.display import Image

In [ ]:
# Autentificar Earth engine
# ee.Authenticate() # Esto solo debe ser ejecutado la primera vez y cada vez que el token expira

ee.Initialize()

In [ ]:
# Definir area de estudio

# Coordenadas
lat = 21.8833333
lon = -102.3

# Punto de interes
poi = ee.Geometry.Point(lon, lat)

# Periodo de tiempo
start_date = "2021-11-01"
end_date = "2022-11-15"

In [ ]:
# Descarga de imagenes Sentinel 2 
sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterBounds(poi)\
    .filterDate(start_date, end_date)

# Cantidad de imagenes obtenidas
print("Imagenes totales:", sentinel.size().getInfo())

In [ ]:
# Informacion de la primera imagen
sentinel.first().getInfo()

In [ ]:
# Nubosidad
print(sentinel.first().get("CLOUDY_PIXEL_PERCENTAGE").getInfo())

In [ ]:
# Porcentaje de agua
print(sentinel.first().get("WATER_PERCENTAGE").getInfo())

In [ ]:
# Fechas
print(datetime.strptime(re.findall("(?<=\_)\\d+(?=T)", sentinel.first().get("DATATAKE_IDENTIFIER").getInfo())[0], "%Y%m%d").date())

In [ ]:
# Bandas 
sentinel.first().bandNames().getInfo()

In [ ]:
# Parametros para preprocesamiento
parameters = {
    "min":0,
    "max":3000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B4", "B3", "B2"] # (R, G, B)
}

In [13]:
# Imagenes en lista
sentinel_list = sentinel.toList(sentinel.size())

# Funcion para mostrar cada imagen
def display_img(ids, parameters, ndvi_display = False):

    for i in ids:
    
        # Fecha de captura
        date = datetime.strptime(re.findall("(?<=\_)\\d+(?=T)", ee.Image(sentinel_list.get(i)).get("DATATAKE_IDENTIFIER").getInfo())[0], "%Y%m%d").date()

        # Nubosidad
        cloud = ee.Image(sentinel_list.get(i)).get("CLOUDY_PIXEL_PERCENTAGE").getInfo()

        # Porcentaje de vegetación
        veg = ee.Image(sentinel_list.get(i)).get("VEGETATION_PERCENTAGE").getInfo()

        # Porcentaje de agua
        water = ee.Image(sentinel_list.get(i)).get("WATER_PERCENTAGE").getInfo()

        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud, "Porcentaje de agua:", water)

        if ndvi_display == True:
            display(Image(url = ee.Image(sentinel_list.get(i)).normalizedDifference(["B8", "B4"]).getThumbURL(parameters)))

        else:
            # Plot
            display(Image(url = ee.Image(sentinel_list.get(i)).getThumbURL(parameters)))
        
# Funcion para crear data frame con info de las imagenes
def img_info(ids):
    data = []
    for i in ids:

        # Fecha de captura
        date = datetime.strptime(re.findall("(?<=\_)\\d+(?=T)", ee.Image(sentinel_list.get(i)).get("DATATAKE_IDENTIFIER").getInfo())[0], "%Y%m%d").date()

        # Nubosidad
        cloud = ee.Image(sentinel_list.get(i)).get("CLOUDY_PIXEL_PERCENTAGE").getInfo()

        # Porcentaje de vegetación
        veg = ee.Image(sentinel_list.get(i)).get("VEGETATION_PERCENTAGE").getInfo()
        
        # Porcentaje de agua
        water = ee.Image(sentinel_list.get(i)).get("WATER_PERCENTAGE").getInfo()

        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud,"Porcentaje de vegetación:", veg, "Porcentaje de agua:", water)
        
        image_data = [i, date, cloud, veg, water]
        data.append(image_data)
    
    return data

In [ ]:
# DF de informacion
sentinel_df = pd.DataFrame(img_info(range(sentinel.size().getInfo())), columns = ["img_id", "date", "cloud_cover", "vegetation_percentage", "water_percentage"])

In [ ]:
# Imagenes recolectadas
parameters = {
    "min":0, # 0 bueno
    "max":3000, # 3000 bueno
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B4", "B3", "B2"] # (R, G, B)
}

display_img([88], parameters = parameters)

In [ ]:
# Data frame con  informacion de las imagenes
sentinel_df.head() # Con esto se puede filtrar por la nubosidad, la cual parece ser buena por debajo de 10

## Filtrado de imagenes

In [ ]:
# Seleccionar las imagenes que tengan una nubosidad por debajo de 10
ids_noClouds = sentinel_df.img_id[sentinel_df.cloud_cover < 10]
len(ids_noClouds) # 90 Imagenes 

In [ ]:
# Definir region de interes
roi = poi.buffer(5000) # Metros = 5 km

In [ ]:
# Nuevos parametros para región de interés
parameters_roi = {
    "min":0,
    "max":3000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B4", "B3", "B2"], # (R, G, B)
    "region":roi # Region de interes
}

In [ ]:
display_img([0], parameters_roi)

In [ ]:
# Imagenes con zoom en la ciudad de Ags
display_img(ids_noClouds, parameters = parameters_roi)

## NDVI

$NDVI = \frac{NIR - RED}{NIR + RED}$

* Saludable = Valores altos
* No saludable = Valores bajos

`ndvi = image.normalizedDifference(["B5", "B4"])`

In [ ]:
# Parametros para el analisis NDVI
palette = ["red", "yellow", "green"]

ndvi_parameters = {
    "min":0, # Este color sera rojo - no saludable
    "max":0.4, # Lo que sea mayor sera verde - saludable
    "dimensions":512,
    "palette":palette,
    "region":roi
}

In [ ]:
# NDVI index por imagen
display_img(ids_noClouds, ndvi_parameters, ndvi_display = True)